In [8]:
import pandas as pd
import numpy as np

In [9]:
# Set some display options for better viewing
pd.set_option('display.max_columns', 100)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# Load your golden dataset
df_raw = pd.read_parquet('../data/processed/nyc_sales_combined.parquet')

# Create the 'borough_name' column for filtering
# Borough codes: 1=Manhattan, 2=Bronx, 3=Brooklyn, 4=Queens, 5=Staten Island
borough_map = {1: 'Manhattan', 2: 'Bronx', 3: 'Brooklyn', 4: 'Queens', 5: 'Staten Island'}
df_raw['borough_name'] = df_raw['borough'].map(borough_map)

# FILTER FOR MANHATTAN ONLY
df_manhattan = df_raw[df_raw['borough_name'] == 'Manhattan'].copy()
print(f"Filtered for Manhattan. New shape: {df_manhattan.shape}")

print("\n--- Manhattan DataFrame Info ---")
df_manhattan.info()

print("\n--- Manhattan DataFrame Head ---")
display(df_manhattan.head())

Filtered for Manhattan. New shape: (126435, 22)

--- Manhattan DataFrame Info ---
<class 'pandas.core.frame.DataFrame'>
Index: 126435 entries, 7664 to 554716
Data columns (total 22 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   borough                         126435 non-null  float64       
 1   neighborhood                    126435 non-null  object        
 2   building_class_category         126435 non-null  object        
 3   tax_class_at_present            126435 non-null  object        
 4   block                           126435 non-null  float64       
 5   lot                             126435 non-null  float64       
 6   easement                        0 non-null       float64       
 7   building_class_at_present       126188 non-null  object        
 8   address                         126435 non-null  object        
 9   apartment_number                126435 non-nu

,borough,neighborhood,building_class_category,tax_class_at_present,block,lot,easement,building_class_at_present,address,apartment_number,zip_code,residential_units,commercial_units,total_units,land_square_feet,gross_square_feet,year_built,tax_class_at_time_of_sale,building_class_at_time_of_sale,sale_price,sale_date,borough_name
7664,1.00,ALPHABET CITY,01 ONE FAMILY DWELLINGS,1,376.00,43.00,NaN,S1,743 EAST 6TH STREET,nan,10009.00,1.00,1.00,2.00,2090.00,3680.00,1940.00,1.00,S1,3200000.00,2019-07-24,Manhattan
7665,1.00,ALPHABET CITY,01 ONE FAMILY DWELLINGS,1,390.00,61.00,NaN,A4,189 EAST 7TH STREET,nan,10009.00,1.00,0.00,1.00,987.00,2183.00,1860.00,1.00,A4,0.00,2019-09-25,Manhattan
7666,1.00,ALPHABET CITY,02 TWO FAMILY DWELLINGS,1,404.00,1.00,NaN,B9,166 AVENUE A,nan,10009.00,2.00,0.00,2.00,1510.00,4520.00,1900.00,1.00,B9,0.00,2019-07-22,Manhattan
7667,1.00,ALPHABET CITY,03 THREE FAMILY DWELLINGS,1,377.00,56.00,NaN,C0,263 EAST 7TH STREET,nan,10009.00,3.00,0.00,3.00,2430.00,3600.00,1899.00,1.00,C0,6300000.00,2019-04-30,Manhattan
7668,1.00,ALPHABET CITY,03 THREE FAMILY DWELLINGS,1,393.00,9.00,NaN,C0,604 EAST 11TH STREET,nan,10009.00,3.00,0.00,3.00,2375.00,5110.00,1939.00,1.00,C0,0.00,2019-10-24,Manhattan


In [10]:
print(df_manhattan['building_class_category'].value_counts().head(20))

building_class_category
13 CONDOS - ELEVATOR APARTMENTS      50607
10 COOPS - ELEVATOR APARTMENTS       43936
17 CONDO COOPS                        6676
09 COOPS - WALKUP APARTMENTS          5458
07 RENTALS - WALKUP APARTMENTS        4073
15 CONDOS - 2-10 UNIT RESIDENTIAL     1962
08 RENTALS - ELEVATOR APARTMENTS      1513
47 CONDO NON-BUSINESS STORAGE         1488
43 CONDO OFFICE BUILDINGS             1166
01 ONE FAMILY DWELLINGS               1061
02 TWO FAMILY DWELLINGS                874
21 OFFICE BUILDINGS                    830
12 CONDOS - WALKUP APARTMENTS          785
22 STORE BUILDINGS                     745
45 CONDO HOTELS                        718
46 CONDO STORE BUILDINGS               705
14 RENTALS - 4-10 UNIT                 619
03 THREE FAMILY DWELLINGS              496
11 SPECIAL CONDO BILLING LOTS          387
31 COMMERCIAL VACANT LAND              365
Name: count, dtype: int64


In [11]:
# Look at the lower percentiles of your raw sale_price data
# Focus on prices that are not zero first
non_zero_prices = df_manhattan[df_manhattan['sale_price'] > 0]['sale_price']
print(non_zero_prices.describe(percentiles=[.01, .05, .10, .25]))

count       101027.00
mean       3901784.99
std       23209420.32
min              1.00
1%             107.28
5%          302235.30
10%         420000.00
25%         670000.00
50%        1205000.00
max     2397501899.00
Name: sale_price, dtype: float64


In [12]:
# Look at the lower percentiles for square footage
non_zero_sqft = df_manhattan[df_manhattan['gross_square_feet'] > 0]['gross_square_feet']
print(non_zero_sqft.describe(percentiles=[.01, .05, .10, .25]))

count     18433.00
mean      29711.52
std      123778.40
min          23.00
1%          400.00
5%          573.00
10%         697.00
25%        1236.00
50%        3808.00
max     8942176.00
Name: gross_square_feet, dtype: float64


In [13]:
# --- Data-Driven Filtering ---

# Justification for Sale Price Filter:
# Our percentile analysis showed that 5% of non-zero sales are below $145,000.
# These are highly unlikely to be arm's-length market transactions (e.g., deed transfers, sales between family).
# We will set a lower bound threshold of $100,000 as a conservative filter to remove these non-market sales.
df_filtered = df_manhattan[df_manhattan['sale_price'] > 100000].copy()
print(f"Shape after price > $100k filter: {df_filtered.shape}")


# Justification for Square Footage Filter:
# Our percentile analysis showed that 5% of properties have listed sqft below 918 sq ft, and the max is over 8 million.
# To ensure we are modeling standard habitable residential units and remove outliers/data errors,
# we will filter for properties between 250 and 20,000 gross square feet.
df_filtered = df_filtered[(df_filtered['gross_square_feet'] > 250) & (df_filtered['gross_square_feet'] < 20000)]
print(f"Shape after square footage filter (250 < sqft < 20k): {df_filtered.shape}")


# --- Select Core Residential Property Types ---

# Justification for Building Class Filter:
# Our model focuses on predicting the value of residential dwellings where people live.
# We are including single-family homes, small multi-family (2-3 units), condos, and co-ops.
# We are explicitly excluding large rental buildings, commercial properties, vacant land,
# and miscellaneous property types (e.g., parking spots) as their valuation drivers are different.
residential_categories = [
    '01 ONE FAMILY DWELLINGS',
    '02 TWO FAMILY DWELLINGS',
    '03 THREE FAMILY DWELLINGS',
    '10 COOPS - ELEVATOR APARTMENTS',
    '13 CONDOS - ELEVATOR APARTMENTS',
    '09 COOPS - WALKUP APARTMENTS',
    '04 TAX CLASS 1 CONDOS', # Often single-family style condos
    '15 CONDOS - 2-10 UNIT RESIDENTIAL',
    '17 CONDO COOPS',
    '12 CONDOS - WALKUP APARTMENTS',
    '16 CONDOS - 2-10 UNIT WITH COMMERCIAL UNIT' # Keep these as they are primarily residential
]

df_residential = df_filtered[df_filtered['building_class_category'].isin(residential_categories)].copy()
print(f"Final shape after filtering for residential categories: {df_residential.shape}")

# --- Final Feature Engineering and Type Conversion ---

# Create an 'age' feature. Replace invalid 'year_built' values (like 0) with NaN first.
df_residential['year_built'] = df_residential['year_built'].replace(0, np.nan)
df_residential['age'] = pd.to_datetime('now').year - df_residential['year_built']

# Create a 'price_per_sqft' feature, which is often a very useful metric
df_residential['price_per_sqft'] = df_residential['sale_price'] / df_residential['gross_square_feet']

# Convert integer-like floats to integers for cleanliness
int_cols = ['zip_code', 'residential_units', 'commercial_units', 'total_units', 'year_built', 'block', 'lot']
for col in int_cols:
    # Use Int64 (capital I) to handle potential missing values (NaNs)
    df_residential[col] = df_residential[col].astype('Int64')
    
# --- Final Inspection ---
print("\n--- Final DataFrame Info ---")
df_residential.info()

print("\n--- Final DataFrame Head ---")
display(df_residential.head())

print("\n--- Final DataFrame Description ---")
display(df_residential[['sale_price', 'gross_square_feet', 'age', 'price_per_sqft']].describe())

Shape after price > $100k filter: (98738, 22)
Shape after square footage filter (250 < sqft < 20k): (10035, 22)
Final shape after filtering for residential categories: (6505, 22)

--- Final DataFrame Info ---
<class 'pandas.core.frame.DataFrame'>
Index: 6505 entries, 7664 to 554468
Data columns (total 24 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   borough                         6505 non-null   float64       
 1   neighborhood                    6505 non-null   object        
 2   building_class_category         6505 non-null   object        
 3   tax_class_at_present            6505 non-null   object        
 4   block                           6505 non-null   Int64         
 5   lot                             6505 non-null   Int64         
 6   easement                        0 non-null      float64       
 7   building_class_at_present       6505 non-null   object        
 8  

,borough,neighborhood,building_class_category,tax_class_at_present,block,lot,easement,building_class_at_present,address,apartment_number,zip_code,residential_units,commercial_units,total_units,land_square_feet,gross_square_feet,year_built,tax_class_at_time_of_sale,building_class_at_time_of_sale,sale_price,sale_date,borough_name,age,price_per_sqft
7664,1.00,ALPHABET CITY,01 ONE FAMILY DWELLINGS,1,376,43,NaN,S1,743 EAST 6TH STREET,nan,10009,1,1,2,2090.00,3680.00,1940,1.00,S1,3200000.00,2019-07-24,Manhattan,85.00,869.57
7667,1.00,ALPHABET CITY,03 THREE FAMILY DWELLINGS,1,377,56,NaN,C0,263 EAST 7TH STREET,nan,10009,3,0,3,2430.00,3600.00,1899,1.00,C0,6300000.00,2019-04-30,Manhattan,126.00,1750.00
7801,1.00,CHELSEA,01 ONE FAMILY DWELLINGS,1,720,6,NaN,A4,483 WEST 22ND STREET,nan,10011,1,0,1,1562.00,4150.00,1901,1.00,A4,6725000.00,2019-09-03,Manhattan,124.00,1620.48
7802,1.00,CHELSEA,01 ONE FAMILY DWELLINGS,1,764,49,NaN,A9,218 WEST 15TH STREET,nan,10011,1,0,1,2141.00,3935.00,1910,1.00,A9,2421900.00,2019-05-22,Manhattan,115.00,615.48
7803,1.00,CHELSEA,01 ONE FAMILY DWELLINGS,1,768,8,NaN,A4,253 WEST 18TH STREET,nan,10011,1,0,1,1521.00,6984.00,1901,1.00,A4,12709140.00,2019-04-11,Manhattan,124.00,1819.75



--- Final DataFrame Description ---


,sale_price,gross_square_feet,age,price_per_sqft
count,6505.00,6505.00,5521.00,6505.00
mean,3831002.25,1981.87,65.45,1756.80
std,5634017.12,1759.59,46.47,1376.07
min,129000.00,277.00,6.00,63.50
25%,1020000.00,783.00,19.00,1196.58
50%,1970000.00,1306.00,52.00,1541.95
75%,4300000.00,2685.00,115.00,2049.18
max,77100000.00,18814.00,225.00,67049.81


In [14]:
output_path = '../data/processed/nyc_sales_analysis_ready.parquet'
df_residential.to_parquet(output_path)

print(f"Successfully saved the analysis-ready dataset with {len(df_residential)} rows to:")
print(output_path)

Successfully saved the analysis-ready dataset with 6505 rows to:
../data/processed/nyc_sales_analysis_ready.parquet
